In [2]:
from numba import njit 
@njit
def mirror(A, f):
    n = A.shape[0]
    m = A.shape[1]
    nlin = A.shape[0] + 2*f
    ncol = A.shape[1] + 2*f
    # Matriz de saída
    B = np.zeros((nlin, ncol))
    # Preeenche miolo
    B[f:nlin-f, f:ncol-f] = A
    # Preenche cantos
    B[0:f, 0:f] = np.flip(A[0:f, 0:f])                          # 1o quadrante
    B[0:f, ncol-f:ncol] = np.flip(A[0:f, m-f:m])                # 2o quadrante
    B[nlin-f:nlin, 0:f] = np.flip(A[n-f:n, 0:f])                # 3o quadrante
    B[nlin-f:nlin, ncol-f:ncol] = np.flip(A[n-f:n, m-f:m])      # 4o quadrante
    # Preenche bordas
    B[0:f, f:ncol-f] = np.flipud(A[0:f, :])             # cima
    B[nlin-f:nlin, f:ncol-f] = np.flipud(A[n-f:n, :])   # baixo
    B[f:nlin-f, 0:f] = np.fliplr(A[:, 0:f])             # esquerda
    B[f:nlin-f, ncol-f:ncol] = np.fliplr(A[:, m-f:m])   # direita
    return B

'''
Non-Local Means padrão (versão rápida)

Parâmetros:

    img: imagem ruidosa de entrada
    h: parâmetro que controla o grau de suavização (quanto maior, mais suaviza)
    f: tamanho do patch (2f + 1 x 2f + 1) -> se f = 3, então patch é 7 x 7
    t: tamanho da janela de busca (2t + 1 x 2t + 1) -> se t = 10, então janela de busca é 21 x 21

'''
@njit
def NLM_fast(img, h, f, t):
    # Dimenssões espaciais da imagem
    m, n = img.shape
    # Cria imagem de saída
    filtrada = np.zeros((m, n))
    # Problema de valor de contorno: replicar bordas
    img_n = mirror(img, f)
    # Loop principal do NLM
    for i in range(m):
        for j in range(n):
            im = i + f;   # compensar a borda adicionada artificialmente
            jn = j + f;   # compensar a borda adicionada artificialmente
            # Obtém o patch ao redor do pixel corrente
            W1 = img_n[im-f:(im+f)+1, jn-f:(jn+f)+1]
            # Calcula as bordas da janela de busca para o pixel corrente (se pixel próximo das bordas, janela de busca é menor)
            rmin = max(im-t, f);  # linha inicial
            rmax = min(im+t, m+f);  # linha final
            smin = max(jn-t, f);  # coluna inicial
            smax = min(jn+t, n+f);  # coluna final
            # Calcula média ponderada
            NL = 0      # valor do pixel corrente filtrado
            Z = 0       # constante normalizadora
            # Loop para todos os pixels da janela de busca
            for r in range(rmin, rmax):
                for s in range(smin, smax):
                    # Obtém o patch ao redor do pixel a ser comparado
                    W2 = img_n[r-f:(r+f)+1, s-f:(s+f)+1]
                    # Calcula o quadrado da distância Euclidiana
                    d2 = np.sum((W1 - W2)*(W1 - W2))
                    # Calcula a medida de similaridade
                    sij = np.exp(-d2/(h**2))               
                    # Atualiza Z e NL
                    Z = Z + sij
                    NL = NL + sij*img_n[r, s]
            # Normalização do pixel filtrado
            filtrada[i, j] = NL/Z
    return filtrada

In [6]:
from skimage.restoration import estimate_sigma
import sys
import numpy as np
import cupy as cp
import skimage
from Utils import (read_directories,save_pickle,save_results_to_xlsx)
from noisy_functions import (add_low_noise_gaussian,add_moderate_noise_gaussian, add_high_noise_gaussian)    
from nlm_functions import (compute_adaptive_q,select_best_h_using_adaptive_q)   

import time


root_dir_output_moderate = f'/workspace/data/output/moderate_noisy'
dir_images_general = f'/workspace/data/input/general_images'

dir_out_nlm = f'{root_dir_output_moderate}/NLM'
dir_out_geonlm =  f'{root_dir_output_moderate}/GEONLM'
dir_out_bm3d =  f'{root_dir_output_moderate}/BM3D'
dir_out_results =  f'{root_dir_output_moderate}/results'


array_dir = read_directories(dir_images_general)

array_nln_moderate_filtereds = []

time_nlm = 0

for file in array_dir:

    file_name = file
    img = skimage.io.imread(f'{dir_images_general}/{file_name}')
    if img.ndim == 4:  # (T, H, W, C) ou (T, H, W)
        img = img[0]

    # Se tiver alpha (RGBA), descarta
    if img.ndim == 3 and img.shape[-1] == 4:
        img = img[..., :3]

    # Converte RGB -> cinza se ainda tiver 3 canais
    if img.ndim == 3 and img.shape[-1] == 3:
        img = skimage.color.rgb2gray(img)  # retorna float em [0,1]

    # >>> NORMALIZAÇÃO CRÍTICA: leve tudo para [0,255] float32 <<<
    if img.dtype.kind == 'f':                     # tipicamente após rgb2gray
        img = (np.clip(img, 0, 1) * 255).astype(np.float32)
    else:
        img = img.astype(np.float32)

    m, n = img.shape

    # Ruído (σ em 0–255, sem alterar seus valores)
    ruidosa = add_moderate_noise_gaussian(img)

    # Clipa para [0,255]
    ruidosa = np.clip(ruidosa, 0, 255)

    # CPU/GPU
    img_noisse_gaussian_np = ruidosa.astype(np.float32)
    estimated_sigma_gaussian_np = estimate_sigma(img_noisse_gaussian_np)  # data_range implícito 255
    img_gpu_noisse_gaussian = np.array(img_noisse_gaussian_np)

    h_nlm = compute_adaptive_q(estimated_sigma_gaussian_np)
    print(f'Imagem: {file_name} | sigma estimado: {estimated_sigma_gaussian_np:.4f} | h_nlm: {h_nlm:.4f}')

     # NLM padrão
    

    ini = time.time()
    
    f=4
    t=7
    filtrada_padrao = NLM_fast(ruidosa, h_nlm, f, t)
    end = time.time()   
    time_nlm +=  end - ini

  
print(f'time_nlm:{time_nlm}')

Imagem: 36.gif | sigma estimado: 11.5011 | h_nlm: 129.0000
Imagem: 39.gif | sigma estimado: 10.4581 | h_nlm: 129.0000
Imagem: 4.gif | sigma estimado: 10.5805 | h_nlm: 129.0000
Imagem: 3.gif | sigma estimado: 10.4300 | h_nlm: 129.0000
Imagem: 32.gif | sigma estimado: 10.8130 | h_nlm: 129.0000
Imagem: 7.gif | sigma estimado: 13.4550 | h_nlm: 129.0000
Imagem: 40.gif | sigma estimado: 10.1813 | h_nlm: 129.0000
Imagem: 0.gif | sigma estimado: 10.5344 | h_nlm: 129.0000
Imagem: 15.gif | sigma estimado: 12.1346 | h_nlm: 129.0000
Imagem: 48.gif | sigma estimado: 10.5074 | h_nlm: 129.0000
Imagem: 38.gif | sigma estimado: 10.4698 | h_nlm: 129.0000
Imagem: 12.gif | sigma estimado: 10.3681 | h_nlm: 129.0000
Imagem: 19.gif | sigma estimado: 10.8112 | h_nlm: 129.0000
Imagem: 41.gif | sigma estimado: 10.5776 | h_nlm: 129.0000
Imagem: 10.gif | sigma estimado: 10.3869 | h_nlm: 129.0000
Imagem: 24.gif | sigma estimado: 10.6578 | h_nlm: 129.0000
Imagem: 28.gif | sigma estimado: 11.2391 | h_nlm: 129.0000
I